In [2]:
%pip install tensorflow==2.2.0
%pip install scikit-learn
%pip install matplotlib
%pip install imutils
%pip install opencv-python
%pip install wget
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wget
url = 'https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/keras-fine-tune-resnet/fine-tune-resnet.zip'
filename = wget.download(url, './') # second argument is the directory to save, default is "."
from zipfile import ZipFile
with ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('.') # argument is the directory to extract_to, default is "."

100% [........................................................................] 93427475 / 93427475

In [4]:
%cd fine-tune-resnet

C:\Users\User\Documents\cnn\fine-tune-resnet\fine-tune-resnet


In [5]:
import kaggle
kaggle.api.authenticate()
kaggle.api.dataset_download_files('imneonizer/normal-vs-camouflage-clothes')

In [6]:
from zipfile import ZipFile
with ZipFile('normal-vs-camouflage-clothes.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [7]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import shutil
import os
import pickle

In [8]:
def plt_imshow(title, image):
	# convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

In [9]:
class Config:
    # initialize the path to the *original* input directory of images
    ORIG_INPUT_DATASET = "8k_normal_vs_camouflage_clothes_images"

    # initialize the base path to the *new* directory that will contain
    # our images after computing the training and testing split
    BASE_PATH = "camo_not_camo"

    # derive the training, validation, and testing directories
    TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
    VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
    TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

    # define the amount of data that will be used training
    TRAIN_SPLIT = 0.75

    # the amount of validation data will be a percentage of the
    # *training* data
    VAL_SPLIT = 0.1

    # define the names of the classes
    CLASSES = ["camouflage_clothes", "normal_clothes"]

    # initialize the initial learning rate, batch size, and number of
    # epochs to train for
    INIT_LR = 1e-4
    BS = 32
    NUM_EPOCHS = 20

    # define the path to checkpoints
    CHECKPOINT_PATH = os.path.sep.join(["output", "checkpoint"])
    
    # define the path to the serialized output model after training
    MODEL_PATH = os.path.sep.join(["output", "model"])
    
    # define the path to training history
    HISTORY_PATH = os.path.sep.join(["output", "history"])

# instantiate a Config object
config = Config()

In [10]:
# grab the paths to all input images in the original input directory
# and shuffle them
imagePaths = list(paths.list_images(config.ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

# compute the training and testing split
i = int(len(imagePaths) * config.TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

# we'll be using part of the training data for validation
i = int(len(trainPaths) * config.VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

# define the datasets that we'll be building
datasets = [
	("training", trainPaths, config.TRAIN_PATH),
	("validation", valPaths, config.VAL_PATH),
	("testing", testPaths, config.TEST_PATH)
]

In [11]:
# loop over the datasets
for (dType, imagePaths, baseOutput) in datasets:
	# show which data split we are creating
	print("[INFO] building '{}' split".format(dType))

	# if the output base output directory does not exist, create it
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)

	# loop over the input image paths
	for inputPath in imagePaths:
		# extract the filename of the input image along with its
		# corresponding class label
		filename = inputPath.split(os.path.sep)[-1]
		label = inputPath.split(os.path.sep)[-2]

		# build the path to the label directory
		labelPath = os.path.sep.join([baseOutput, label])

		# if the label output directory does not exist, create it
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)

		# construct the path to the destination image and then copy
		# the image itself
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] 'creating camo_not_camo\training' directory
[INFO] 'creating camo_not_camo\training\camouflage_clothes' directory
[INFO] 'creating camo_not_camo\training\normal_clothes' directory
[INFO] building 'validation' split
[INFO] 'creating camo_not_camo\validation' directory
[INFO] 'creating camo_not_camo\validation\normal_clothes' directory
[INFO] 'creating camo_not_camo\validation\camouflage_clothes' directory
[INFO] building 'testing' split
[INFO] 'creating camo_not_camo\testing' directory
[INFO] 'creating camo_not_camo\testing\camouflage_clothes' directory
[INFO] 'creating camo_not_camo\testing\normal_clothes' directory


In [12]:
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--plot", type=str, default="plot.png",
#	help="path to output loss/accuracy plot")
#args = vars(ap.parse_args())

# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"plot": "plot.png"
}

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))
print(list(paths.list_images(config.VAL_PATH)))

['camo_not_camo\\validation\\camouflage_clothes\\00010025.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\00010046.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\00010095.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\00010144.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001015.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\00010150.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\000108.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001115.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001118.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001120.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001176.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001191.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001216.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001222.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\0001225.jpg', 'camo_not_camo\\validation\\camouflage_clothes\\000123.jpg', 'cam

In [13]:
# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=25,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [14]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BS)

Found 10731 images belonging to 2 classes.
Found 1192 images belonging to 2 classes.
Found 3975 images belonging to 2 classes.


In [15]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

[INFO] preparing model...


In [16]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
model_checkpoint_callback = ModelCheckpoint(os.path.sep.join([config.CHECKPOINT_PATH, filepath]), monitor='val_accuracy',verbose=1, 
                            save_best_only=True)

In [17]:
# compile the model
opt = Adam(lr=config.INIT_LR, decay=config.INIT_LR / config.NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the model
print("[INFO] training model...")
history = model.fit(
	trainGen,
	steps_per_epoch=totalTrain // config.BS,
	validation_data=valGen,
	validation_steps=totalVal // config.BS,
	epochs=config.NUM_EPOCHS,
    callbacks=[model_checkpoint_callback],
    initial_epoch=0)

[INFO] training model...
Epoch 1/20
 23/335 [=>............................] - ETA: 4:34 - loss: 0.5723 - accuracy: 0.7541

KeyboardInterrupt: 

In [ ]:
# serialize the model to disk
print("[INFO] saving model...")
model.save(config.MODEL_PATH, save_format="h5")

In [23]:
class History_trained_model(object):
    def __init__(self, history, epoch, params):
        self.history = history
        self.epoch = epoch
        self.params = params

In [ ]:
with open(os.path.sep.join([config.HISTORY_PATH, "train_history_dict"]), 'wb') as file_pi:
    model_history= History_trained_model(history.history, history.epoch, history.params)
    pickle.dump(model_history, file_pi, pickle.HIGHEST_PROTOCOL)

## Below is for loading and using models and history

In [18]:
new_model = load_model(os.path.sep.join([config.CHECKPOINT_PATH, 'weights-improvement-01-0.99.hdf5']))
# new_model = load_model(config.MODEL_PATH)

In [19]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
def load_image(img_path, show=False):

    img = load_img(img_path, target_size=(224, 224))
    img_tensor = img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    # img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor


# load the image
image_path = os.path.sep.join([config.BASE_PATH, "testing", "normal_clothes", "0001259.jpg"])
img = load_image(image_path)
my_predIdxs = new_model.predict(img)
print(my_predIdxs)
print(np.argmax(my_predIdxs, axis=1))

[[2.7791271e-05 9.9997222e-01]]
[1]


In [25]:
with open(os.path.sep.join([config.HISTORY_PATH, "train_history_dict"]), 'rb') as file_pi:
    new_history = pickle.load(file_pi)
print(new_history.history)

{'loss': [0.05678858235478401, 0.054267339408397675, 0.04310678690671921, 0.04192599654197693, 0.04007718339562416, 0.036789726465940475, 0.034828510135412216, 0.031422946602106094, 0.02664753422141075, 0.029027733951807022, 0.027005324140191078, 0.026380838826298714, 0.024390211328864098, 0.02230473980307579, 0.02499152347445488, 0.021458376199007034, 0.021539272740483284, 0.02122008427977562, 0.019408682361245155, 0.017830194905400276], 'accuracy': [0.9804654717445374, 0.9821478724479675, 0.9840171933174133, 0.9854192137718201, 0.9865407943725586, 0.9864473342895508, 0.9872885346412659, 0.9896252155303955, 0.9906533360481262, 0.9892513155937195, 0.9899990558624268, 0.9900925159454346, 0.9917749166488647, 0.9916814565658569, 0.992055356502533, 0.9928030371665955, 0.992522656917572, 0.992522656917572, 0.9930834770202637, 0.9936442375183105], 'val_loss': [0.04013579338788986, 0.03811473399400711, 0.03790886700153351, 0.03639049828052521, 0.03454349935054779, 0.03610082343220711, 0.02848